In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup: Connectivity

In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  Delete all graphs

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  Delete all databases

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  Create database

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  Create a graph

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:

#  Connect to graph

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Step 01:  Load from CSV

In [2]:

import pandas as pd
import numpy as np

print("--")


--


In [5]:

#  Load a DataFrame from CSV, Nodes/Vertices


l_InputFile  = "./10_N_MovieDB/24_nodes.txt"

df_all_nodes1 = pd.read_csv(
   l_InputFile,
   delimiter = ",",
   dtype = {
      # "id"        : int,
      "id"        : np.dtype('O'),
      "_labels"   : np.dtype('O'),
      # "born"      : float, 
      "born"      : np.dtype('O'),
      "name"      : np.dtype('O'),
      # "released"  : float,
      "released"  : np.dtype('O'),
      "tagline"   : np.dtype('O'),
      "title"     : np.dtype('O')
      })

print("Number of rows: %d" % (len(df_all_nodes1)) )
   #
print("--")


Number of rows: 171
--


In [10]:

#  Column renames

df_all_nodes2 = df_all_nodes1.rename(columns={"_id": "id", "_labels": "label"})

#  Change float values to integer, remove leading semicolon from label

df_all_nodes3 = df_all_nodes2.assign(
   # born     = lambda x: x.born.fillna(0.0).astype(int), 
   born     = lambda x: x.born.fillna(0.0).astype(str), 
   # id       = lambda x: x.id.fillna(0.0).astype(int),
   id       = lambda x: x.id.fillna(0.0).astype(str),
   # released = lambda x: x.released.fillna(0.0).astype(int),
   released = lambda x: x.released.fillna(0.0).astype(str),
   label    = lambda x: x.label.astype(str).str[1:]
   )


print("Number of rows: %d" % (len(df_all_nodes1)) )
print("")
display(df_all_nodes3.head(5))

print("--")


Number of rows: 171



,id,label,born,name,released,tagline,title
0,0.0,Movie,0.0,NaN,1999.0,Welcome to the Real World,The Matrix
1,1.0,Person,1964.0,Keanu Reeves,0.0,NaN,NaN
2,2.0,Person,1967.0,Carrie-Anne Moss,0.0,NaN,NaN
3,3.0,Person,1961.0,Laurence Fishburne,0.0,NaN,NaN
4,4.0,Person,1960.0,Hugo Weaving,0.0,NaN,NaN


--


# Step 02: Make specific DataFrames

In [ ]:

#  Now, populate Persons and Movies

df_persons = df_all_nodes3[df_all_nodes3["label"] == "Person"][["id", "label", "born", "name"]]

df_movies  = df_all_nodes3[df_all_nodes3["label"] == "Movie"][["id", "label", "released", "tagline", "title"]]


display(len(df_persons))
display(df_persons.head(10))
display(len(df_movies))
display(df_movies.head(10))


print("--")


In [ ]:

display(len(df_persons))
display(df_persons.head(10))
display(len(df_movies))
display(df_movies.head(10))


In [ ]:
# ##################################################################
#
#  Repeat the style of work from above, but now for edges
#
#  .  Some of the property names have a leading underscore. Change those.
#  .  Some values which should be integer, are float.


In [ ]:
#  Load a DataFrame from CSV, Edges

l_InputFile  = "./10_NMovieDB/25_edges.txt"

df_all_edges1 = dd.read_csv(
   l_InputFile,
   delimiter = ",",
   dtype = {
      # "_start"    : float,
      "_start"    : np.dtype('O'),
      # "_end"      : float, 
      "_end"      : np.dtype('O'),
      "_type"     : np.dtype('O'),
      # "rating"    : float,
      "rating"    : np.dtype('O'),
      "roles"     : np.dtype('O'),
      "summary"   : np.dtype('O')
      })

print("--")


In [ ]:
display(len(df_all_edges1))
display(df_all_edges1.head(10))


In [ ]:
df_all_edges2 = df_all_edges1.rename(columns={"_start": "START_ID", "_end": "END_ID", "_type": "TYPE",
   "rating": "RATING", "roles": "ROLES", "summary": "SUMMARY"})

df_all_edges3 = df_all_edges2.assign(
   # START_ID = lambda x: x.START_ID.fillna(0.0).astype(int), 
   START_ID = lambda x: x.START_ID.fillna(0.0).astype(str), 
   # END_ID   = lambda x: x.END_ID.fillna(0.0).astype(int),
   END_ID   = lambda x: x.END_ID.fillna(0.0).astype(str),
   # RATING   = lambda x: x.RATING.fillna(0.0).astype(int)
   RATING   = lambda x: x.RATING.fillna(0.0).astype(str)
   )

print("--")


In [ ]:
display(len(df_all_edges3))
display(df_all_edges3.head(10))


In [ ]:
#  Split out the various edges

df_reviewed = df_all_edges3[df_all_edges3["TYPE"] == "REVIEWED"][["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]]

df_wrote    = df_all_edges3[df_all_edges3["TYPE"] == "WROTE"   ][["START_ID", "END_ID", "TYPE"]]

df_produced = df_all_edges3[df_all_edges3["TYPE"] == "PRODUCED"][["START_ID", "END_ID", "TYPE"]]

df_directed = df_all_edges3[df_all_edges3["TYPE"] == "DIRECTED"][["START_ID", "END_ID", "TYPE"]]

df_follows  = df_all_edges3[df_all_edges3["TYPE"] == "FOLLOWS" ][["START_ID", "END_ID", "TYPE"]]

#  "roles" is a string similar to  '[ "a", "b", "c" ]'
#
#  This was automatically coming in as a list-
#  Cool

df_actedin  = df_all_edges3[df_all_edges3["TYPE"] == "ACTED_IN"][["START_ID", "END_ID", "TYPE", "ROLES"]]

print("--")


In [ ]:

display(len(df_reviewed))
display(df_reviewed.head(2))

display(len(df_wrote))
display(df_wrote.head(2))

display(len(df_produced))
display(df_produced.head(2))

display(len(df_directed))
display(df_directed.head(2))

display(len(df_follows))
display(df_follows.head(2))

display(len(df_actedin))
display(df_actedin.head(2))


In [ ]:
# ##################################################################

In [ ]:
#  Didn't need this; also don't know if it had any effect

# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=2)

# print("--")

In [ ]:
# ##################################################################
#
#  Finally now, load the vertices/nodes into a graph
#
#  Some hinkiness we need to work around ..
#
#     .  The Dask DataFrames here were loaded from CSV, and those CSV
#        files were found, in scope.
#        The KG DataFrame importer will reference that same file
#        pathname, and the file will not be in scope. Basically,
#        it was expected that these files be on S3/GS all along.
#        I hate to have that dependency because, just one more thing
#        to have to manage.
#
#     .  Pandas DataFrames do not suffer this grief.
#        So, convert the Dask DataFrames to Pandas.


#  This did not bear fruit; convert the Dask DataFrame to a Dask Array,
#  and back.

#  da_persons    = df_persons.to_dask_array()
#  df_persons2   = dd.io.from_dask_array(da_persons,  columns=["id", "label", "born", "name"])
#  df_persons2.compute()


df_persons2    = df_persons.compute()
df_movies2     = df_movies.compute()
   #
df_directed2   = df_directed.compute()
df_reviewed2   = df_reviewed.compute()
df_wrote2      = df_wrote.compute()
df_produced2   = df_produced.compute()
df_follows2    = df_follows.compute()
df_actedin2    = df_actedin.compute()

print("--")


In [ ]:
from katana_enterprise.remote import import_data

print("--")


In [ ]:

with import_data.DataFrameImporter(my_graph) as df_importer:
    
   # Person
   #
   df_importer.nodes_dataframe(
      df_persons2[["id", "label", "born", "name"]],
      id_column  = "id",
      id_space   = "Person"
      )
   #  Movie
   #
   df_importer.nodes_dataframe(
      df_movies2[["id", "label", "title", "tagline"]],
      id_column  = "id",
      id_space   = "Movie"
      )  
    
   #  DIRECTED
   #
   df_importer.edges_dataframe(
      df_directed2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "DIRECTED"
      )
   #  REVIEWED
   #
   df_importer.edges_dataframe(
      df_reviewed2[["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "REVIEWED"
      )
   #  WROTE
   #
   df_importer.edges_dataframe(
      df_wrote2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "WROTE"
      )
   #  PRODUCED
   #
   df_importer.edges_dataframe(
      df_produced2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "PRODUCED"
      )
   #  FOLLOWS
   #
   df_importer.edges_dataframe(
      df_follows2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "FOLLOWS"
      )
   #  ACTEDIN
   #
   df_importer.edges_dataframe(
      df_actedin2[["START_ID", "END_ID", "TYPE", "ROLES"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "ACTEDIN"
      )
    

   df_importer.insert()
    
    

In [ ]:

l_result1 = my_graph.query("""

   MATCH (a) 
   WITH DISTINCT LABELS(a) AS temp, COUNT(a) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """)

display(print(l_result1))


In [ ]:

l_result1 = my_graph.query("""

   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """)

display(print(l_result1))


In [ ]:

result = my_graph.query("""

   MATCH (x) -[r]-> (a)
   RETURN x, r AS rel, a
   
   """,
   contextualize=True)

result.view()


# Output a graph as a a local file

In [ ]:

#  Formatting could use a little work, but the concept is here ..

l_result1 = my_graph.query("""
   MATCH (n: Person) 
   RETURN n
   """)
      #
# display(print(l_result1))

l_result2 = my_graph.query("""
   MATCH (n) - [r: WROTE] -> (m)
   RETURN r
   """)
      #
# display(print(l_result2))


In [ ]:

l_nodes = []
   #
for l_node in l_result1.itertuples():
   l_nodes.append(l_node)

l_file = open("nodes.txt", "w")
l_file.write(str(l_nodes))
l_file.close()


l_edges = []
   #
for l_edge in l_result2.itertuples():
   l_edges.append(l_edge)

l_file = open("edges.txt", "w")
l_file.write(str(l_edges))
l_file.close()


print("--")
